In [11]:
import pandas as pd

In [12]:
data1 = pd.read_csv('data.csv')
data1.fillna(0)
data1.head()

,install_date,publisher_name,impressions,clicks,installs,cost,revenue_d7
0,2022-08-07,Wehear - Audiobooks & Stories,1,1,0,0.0,0.0
1,2022-08-07,Bump Pop,13,5,0,0.0,0.0
2,2022-08-07,Punch Guys,6,0,0,0.0,0.0
3,2022-08-07,Spot The Hidden Differences,1,0,0,0.0,0.0
4,2022-08-07,Idle Alligator-Reptile Empire,1,0,0,0.0,0.0


In [13]:
g_data = data1.groupby(['publisher_name', 'install_date']).first().reset_index()
g_data.head(20)

,publisher_name,install_date,impressions,clicks,installs,cost,revenue_d7
0,#1 Piggy : Chapter One,2022-08-08,1,1,0,0.0,0.000000
1,#1 Piggy : Chapter One,2022-08-13,2,2,0,0.0,0.000000
2,(BCDS)(AS) Bingo Country Days,2022-09-02,1,0,0,0.0,0.000000
3,(C20 iOS) Slots - casino slot machines,2022-08-12,1,0,0,0.0,0.000000
4,(C20 iOS) Slots - casino slot machines,2022-08-15,1,1,0,0.0,0.000000
5,(C20 iOS) Slots - casino slot machines,2022-08-16,1,0,0,0.0,0.000000
6,(COBS)(AS) Bingo Country Boys,2022-08-12,1,0,0,0.0,0.000000
7,(DFTL - iOS) Don't Forget the Lyrics,2022-08-09,1,1,0,0.0,0.000000
8,(DFTL - iOS) Don't Forget the Lyrics,2022-08-10,7,4,0,0.0,0.000000
9,(DFTL - iOS) Don't Forget the Lyrics,2022-08-11,8,3,1,0.8,3.400113


In [14]:
#check_res = g_data[g_data['publisher_name'] == "(iOS) - BJ1 - Blackjack 21! Casino Card Game"]
#check_res.head(50)

In [15]:
#check_res1 = check_res[check_res['cost'] != 0]
#check_res1.head()

In [16]:
data = g_data[['publisher_name', 'install_date', 'cost', 'revenue_d7']]

In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
data['install_date'] = pd.to_datetime(data['install_date'])

publishers_grouped = data.groupby('publisher_name')

results = pd.DataFrame(columns=['publisher_name', 'status'])

for publisher, publisher_data in publishers_grouped:
    last_install_date = publisher_data['install_date'].max()
    
    if (last_install_date - publisher_data['install_date'].min()).days < 7:
        results.loc[len(results)] = [publisher, 'not enough days']
        continue

    current_date = publisher_data['install_date'].min()
    while current_date < last_install_date:
        next_7_days_data = publisher_data[(publisher_data['install_date'] >= current_date) & 
                                           (publisher_data['install_date'] < current_date + pd.Timedelta(days=7))]
        
        if next_7_days_data['revenue_d7'].sum() / next_7_days_data['cost'].sum() > 0.05:
            current_date += pd.Timedelta(days=7)
            continue
        
        next_7_days_cost_sum = publisher_data[(publisher_data['install_date'] >= current_date + pd.Timedelta(days=7)) & 
                                               (publisher_data['install_date'] < current_date + pd.Timedelta(days=14))]['cost'].sum()
        if next_7_days_cost_sum != 0:
            results.loc[len(results)] = [publisher, 'blocked']
            current_date += pd.Timedelta(days=7)
        else:
            current_date += pd.Timedelta(days=7)
    
    if current_date >= last_install_date:
        if 'blocked' not in results[results['publisher_name'] == publisher]['status'].values and 'not enough days' not in results[results['publisher_name'] == publisher]['status'].values:
            results.loc[len(results)] = [publisher, 'valid']
    else:
        results.loc[len(results)] = [publisher, 'not enough days']

results.head(20)


C:\Users\asus\AppData\Local\Temp\ipykernel_16416\1281245020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['install_date'] = pd.to_datetime(data['install_date'])


,publisher_name,status
0,#1 Piggy : Chapter One,not enough days
1,(BCDS)(AS) Bingo Country Days,not enough days
2,(C20 iOS) Slots - casino slot machines,not enough days
3,(COBS)(AS) Bingo Country Boys,not enough days
4,(DFTL - iOS) Don't Forget the Lyrics,valid
5,(Dino) Monster Horror: Scary Playtime (MAX),not enough days
6,(Dino) Super Stickman Hero 3D (MAX),not enough days
7,(Dino)(SpiderRopeHero)Amazing Flying Hero(MAX),valid
8,(Dino)Draw The Missing Part(MAX),not enough days
9,(HAM iOS) Heroes Magic War,valid


In [18]:
cleaned_results = results.dropna().drop_duplicates()
cleaned_results.shape

(8165, 2)

In [19]:
blocked_results = cleaned_results[cleaned_results['status'] == 'blocked']
blocked_results.shape

(1184, 2)

In [20]:
cleaned_results.to_csv('status.csv', index=False)